# Assignment 1 : Analysing Flight-delays Data

Name : Pichaphop Sunthornjittanon

Student ID : 31258301

# Table of contents<a class="anchor" id="table"></a>

* [1 Working with RDD](#1)
* [1.1 Data Preparation and Loading](#1.1)
* [1.1.1 Creating SparkSession & SparkContext](#OneOneOne)
* [1.1.2 Read CSV files, Preprocessing, and final(formatted data) RDD for each file](#OneOneTwo)
* [1.1.2.1 Flights RDD](#1.1.2.1)
* [1.1.2.2 Airports RDD](#1.1.2.2)
* [1.1.3 Show RDD number of columns, and number of records](#1.1.3)
* [1.2 Dataset flights partitioning](#1.2)
* [1.2.1 Obtain the maximum arrival time ](#1.2.1)
* [1.2.2 Obtain the maximum minimum time ](#1.2.2)
* [1.2.3 Define hash partitioning](#1.2.3)
* [1.2.4 Display the records in each partition](#1.2.4)
* [1.3 Query RDD](#1.3)
* [1.3.1 Collect a total number of flights for each month for all flights](#1.3.1)
* [1.3.2 Collect the average delay for each month for all flights](#1.3.2)
* [2 Working with DataFrames](#2)
* [2.1 Data Preparation and Loading](#2.1)
* [2.1.1 Define DataFrames](#2.1.1)
* [2.1.2 Display the Scheme of DataFrames](#2.1.2)
* [2.1.3 Transform date-time and location column](#2.1.3)
* [2.2.1 January Flights Events with ANC airport](#2.2.1)
* [2.2.2 Average Arrival Delay From Origin to Destination](#2.2.2)
* [2.2.3 Join Query with Airports DataFrame](#2.2.3)
* [2.3 Analysis](#2.3.1)
* [2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights](#2.3.1)
* [2.3.2 Display mean arrival delay each month](#2.3.2)
* [2.3.3 Relationship between mean departure delay and mean arrival delay](#2.3.3)
* [3 RDDs vs DataFrame vs Spark SQL](#3)
* [3.1 RDD Operation](#3.1)
* [3.2 DataFrame Operation](#3.1)
* [3.3 Spark SQL Operation](#3.1)
* [3.4 Discussion](#3.1)


# 1 Working with RDD<a class="anchor" id="1"></a>

In this section, you will need to create RDDs from the given datasets, perform partitioning in
these RDDs and use various RDD operations to answer the queries for crash analysis.

## 1.1 Data Preparation and Loading<a class="anchor" id="1.1"></a>
### 1.1.1 Create SparkSession and SparkContext<a class="anchor" id="OneOneOne"></a>
[Back to top](#table)



1. Write the code to create a SparkContext object using SparkSession, which tells Spark
how to access a cluster. To create a SparkSession you first need to build a SparkConf
object that contains information about your application. Give an appropriate name for
your application and run Spark locally with as many working processors as logical
cores on your machine.

In [1]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
master = "local[*]"


# Assign the name to be shown on the Spark cluster UI page
app_name = "Analysing Flight Delay Data"

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL


# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

#  Create SparkContext object using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')


### 1.1.2 Import CSV files and Make RDD for each file<a class="anchor" id="OneOneTwo"></a>
[Back to top](#table)

2. Read the 20 files of “flight*.csv” file into a single RDD (flights_rdd) and
“airports.csv” file into a single RDD (airports_rdd). For each RDD, remove the header
row and parse each comma-delimited line into a Row object with each columnfollowing the data type given in the jupyter notebook cell. Please convert some
columns into the preferred format. Columns that should be converted into integer :
'YEAR', 'MONTH', 'DAY','DAY_OF_WEEK', 'FLIGHT_NUMBER'. Column that
should be converted into float data type : 'DEPARTURE_DELAY',
'ARRIVAL_DELAY', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN', and
'TAXI_OUT'. While the rest are kept as string format. Note that in this preprocessing
task, you are asked to build a set of functions which load the csv data into the RDD
object, remove the header of the RDD object, and finally parse the RDD object into
the desired format (integer, float, or string).

#### 1.1.2.1 Flights RDD <a class="anchor" id="1.1.2.1"></a>
[Back to top](#table)

In [2]:

# 1.1.2.1 Flights RDD

# a.import all csv file that start with flight
flight_rdd = sc.textFile('data/flight*.csv')

# b. Split each line separated by comma into a list 
flight_rdd = flight_rdd.map(lambda line: line.split(','))

# c. Remove the header
flight_header = flight_rdd.first() 
flight_rdd = flight_rdd.filter(lambda x: x != flight_header)

# d. Convert data type 
flight_rdd = flight_rdd.map(lambda x: [int(x[0] or 0),int(x[1] or 0),int(x[2] or 0),int(x[3] or 0),x[4],int(x[5] or 0),
                                      x[6],x[7],x[8],x[9],x[10],float(x[11] or 0),
                                      float(x[12]or 0),x[13],x[14],float(x[15]or 0),float(x[16]or 0),float(x[17]or 0),
                                      x[18],float(x[19]or 0),x[20],x[21],float(x[22]or 0),x[23],
                                      x[24],x[25],x[26],x[27],x[28],x[29],x[30]])


# print("Number of record :",flight_rdd.count())
# flight_rdd.take(1000)

#### 1.1.2.2 Airports RDD <a class="anchor" id="1.1.2.2"></a>
[Back to top](#table)

In [3]:
# 1.1.2.2 Airports RDD

# a. import all csv file that start with airport
airport_rdd = sc.textFile('data/airports.csv')

# b. Split each line separated by comma into a list 
airport_rdd = airport_rdd.map(lambda line: line.split(','))

# c. Remove the header
airport_header = airport_rdd.first() 
airport_rdd = airport_rdd.filter(lambda x: x != airport_header)

# print("Number of record :",airport_rdd.count())
# airport_rdd.take(5)

### 1.1.3 Show RDD number of columns, and number of records <a class="anchor" id="1.1.3"></a>
[Back to top](#table)

3. For each RDD, display the number of columns, the total number of records, and
display the number of partitions.

In [4]:
print("-------Flight----------")
print("Number of records for flight :",flight_rdd.count())
print("Number of columns for flight :", len(flight_header))
print("Number of partition :", flight_rdd.getNumPartitions())

print("-------Airport----------")
print("Number of records for airport:",airport_rdd.count())
print("Number of columns for airport :", len(airport_header))
print("Number of partition :", airport_rdd.getNumPartitions())

-------Flight----------
Number of records for flight : 582184
Number of columns for flight : 31
Number of partition : 20
-------Airport----------
Number of records for airport: 322
Number of columns for airport : 7
Number of partition : 2


## 1.2 Dataset Partitioning <a class="anchor" id="1.2"></a>

[Back to top](#table)

In this section, you will need to analyse the RDD partitions.

● How many partitions do the above RDDs have?

● How is the data in these RDDs partitioned by default, when we do not explicitly specify any partitioning strategy?

In [5]:
from pyspark.rdd import RDD

#A Function to print the data items in each RDD

def print_partitions(data):
    if isinstance(data, RDD):
        numPartitions = data.getNumPartitions()
        partitions = data.glom().collect()
    else:
        numPartitions = data.rdd.getNumPartitions()
        partitions = data.rdd.glom().collect()
    
    print(f"####### NUMBER OF PARTITIONS: {numPartitions}")
    for index, partition in enumerate(partitions):
        # show partition if it is not empty
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records")


In [6]:
print('################## FLIGHT ##################')
print_partitions(flight_rdd)

print('################## AIRPORT ##################')
print_partitions(airport_rdd)

################## FLIGHT ##################
####### NUMBER OF PARTITIONS: 20
Partition 0: 31221 records
Partition 1: 30803 records
Partition 2: 28472 records
Partition 3: 30863 records
Partition 4: 28471 records
Partition 5: 28532 records
Partition 6: 27039 records
Partition 7: 30535 records
Partition 8: 27234 records
Partition 9: 29452 records
Partition 10: 28615 records
Partition 11: 30102 records
Partition 12: 28940 records
Partition 13: 28081 records
Partition 14: 29844 records
Partition 15: 27545 records
Partition 16: 30542 records
Partition 17: 24994 records
Partition 18: 31056 records
Partition 19: 29843 records
################## AIRPORT ##################
####### NUMBER OF PARTITIONS: 2
Partition 0: 162 records
Partition 1: 160 records


#### Answer

- How many partitions do the above RDDs have?

flight_rdd has 20 partitions and airport_rdd has 2 partition

- How is the data in these RDDs partitioned by default, when we do not explicitly specify any partitioning strategy?

By deafault, Spark uses Random Equal Partitioning

Once flights_rdd is created in 1.1.2, we note that the ‘ARRIVAL_DELAY’ column has been
converted into a float data type. This column represents the gap between arrival time and the
scheduled time represented in the column 'ARRIVAL_TIME' and 'SCHEDULED_ARRIVAL'
respectively. Negative value means that the arrival time is earlier than scheduled time and
vice versa.

### 1.2.1 Obtain the maximum arrival time <a class="anchor" id="1.2.1"></a>
[Back to top](#table)

1. Obtain the maximum arrival delay using RDD from flights_rdd (could be a positive
value).

In [7]:
# Print the maximum arrival delay
print('The maximum arrival delay is',flight_rdd.max(key = lambda x: x[22])[22])

The maximum arrival delay is 1665.0


### 1.2.2 Obtain the minimum arrival time <a class="anchor" id="1.2.2"></a>
[Back to top](#table)

2. Obtain the minimum arrival delay using RDD from flights_rdd (could be a negative
value).

In [8]:
# Print the minimum arrival delay
print('The minimum arrival delay is',flight_rdd.min(key = lambda x: x[22])[22])

The minimum arrival delay is -82.0


### 1.2.3 Define hash partitioning function <a class="anchor" id="1.2.3"></a>
[Back to top](#table)

Make a python function and define a hash partitioning function

In [9]:
# Define Hash function
def hash_function(key):
    total = 0
    key = abs(int(key))
    for digit in str(key):
        total += int(digit)
    return total

no_of_partitions=20

In [10]:
# Transform data to include key(arrival delay) used for hash partioning  
flight_hash_rdd = flight_rdd.map(lambda x: (x[22],[int(x[0] or 0),int(x[1] or 0),int(x[2] or 0),int(x[3] or 0),x[4],int(x[5] or 0),
                                      x[6],x[7],x[8],x[9],x[10],float(x[11] or 0),
                                      float(x[12]or 0),x[13],x[14],float(x[15]or 0),float(x[16]or 0),float(x[17]or 0),
                                      x[18],float(x[19]or 0),x[20],x[21],float(x[22]or 0),x[23],
                                      x[24],x[25],x[26],x[27],x[28],x[29],x[30]]))

In [11]:
# Create hash_partitioned_rdd used hash partioning defined above
hash_partitioned_rdd = flight_hash_rdd.partitionBy(no_of_partitions, hash_function)

### 1.2.4 Display the records in each partition <a class="anchor" id="1.2.4"></a>
[Back to top](#table)

In [12]:
# Display the records in each partition
print_partitions(hash_partitioned_rdd) 

####### NUMBER OF PARTITIONS: 20
Partition 0: 23150 records
Partition 1: 47995 records
Partition 2: 59849 records
Partition 3: 61278 records
Partition 4: 61046 records
Partition 5: 59210 records
Partition 6: 56957 records
Partition 7: 55308 records
Partition 8: 53121 records
Partition 9: 50535 records
Partition 10: 24699 records
Partition 11: 10902 records
Partition 12: 6160 records
Partition 13: 4023 records
Partition 14: 2955 records
Partition 15: 2107 records
Partition 16: 1472 records
Partition 17: 881 records
Partition 18: 434 records
Partition 19: 102 records


#### Give an argument about number of record in each partition

- Con of Hash partioning compare to the default strategy (Random Equal Partitioning):

Skewed distribution can happen shown in the cell above

- Pro of Hash partioning compare to the default strategy (Random Equal Partitioning):

Group data in semantical way so if we define a good hash function, it will improve the performance


## 1.3 Query RDD  <a class="anchor" id="1.3"></a>
### 1.3.1 Collect a total number of flights for each month <a class="anchor" id="1.3.1"></a>
[Back to top](#table)



In [13]:
# Use map to create tuple that contains month in the first element and 1 in the second (use for count)
total_flights_each_month_rdd = flight_rdd.map(lambda x: (x[1],1))

# Reduce by key to group each month together and sum 1 acted as count
total_flights_each_month_rdd = total_flights_each_month_rdd.reduceByKey(lambda x,y: x+y)

# Show the result
total_flights_each_month_rdd.take(12)

[(1, 47136),
 (2, 42798),
 (3, 50816),
 (4, 48810),
 (5, 49691),
 (6, 50256),
 (7, 52065),
 (8, 50524),
 (9, 46733),
 (10, 48680),
 (11, 46809),
 (12, 47866)]

### 1.3.2 Collect the average delay for each month <a class="anchor" id="1.3.2"></a>
[Back to top](#table)

In [14]:
# Use map to create tuple that contains month in the first element and 
# tuple of arrival delay and 1 (use for count) in the second 
average_delay_each_month_rdd = flight_rdd.map(lambda x: (x[1],(x[22],1)))

# Reduce by key to group each month together, sum 1 acted as count and sum  total arrival delay
average_delay_each_month_rdd = average_delay_each_month_rdd.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))

# Calculate average delay using total arrival delay divided by total flights
average_delay_each_month_rdd = average_delay_each_month_rdd.mapValues(lambda x : x[0] / x[1])

# Show the result
average_delay_each_month_rdd.take(12)

[(1, 5.652155465037339),
 (2, 7.722627225571288),
 (3, 4.889286838790932),
 (4, 3.1355050194632246),
 (5, 4.644402406874485),
 (6, 9.534662527857371),
 (7, 6.701373283395755),
 (8, 4.652501781331645),
 (9, -0.8448847709327456),
 (10, -0.5383935907970419),
 (11, 0.8206114208805999),
 (12, 6.035244223457151)]

# 2 Working with DataFrame <a class="anchor" id="2"></a>
## 2.1. Data Preparation and Loading <a class="anchor" id="2.1"></a>
### 2.1.1 Define dataframes and loading scheme<a class="anchor" id="2.1.1"></a>
[Back to top](#table)

1. Load all flights and airports data into two separate dataframes. Name the dataframes
as flightsDf and airportsDf respectively. Hint : use the module
spark.read.format(“csv”), with header option is true and inferSchema is true.

In [15]:
# Load flight data
flightsDf = spark.read.format('csv').option('header',True).option('inferSchema',True).load('data/flight*.csv')

In [16]:
# Load airport data
airportsDf = spark.read.format('csv').option('header',True).option('inferSchema',True).load('data/airport*.csv')

### 2.1.2 Display the schema of the final two dataframes<a class="anchor" id="2.1.2"></a>
[Back to top](#table)

2. Display the schema of the final of two dataframes.

In [17]:
# Display flight schema
flightsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [18]:
# Display airport schema
airportsDf.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



## 2.2. Query Analysis <a class="anchor" id="2.2"></a>

Implement the following queries using dataframes. You need to be able to perform operations
like filtering, sorting, joining and group by using the functions provided by the DataFrame
API.

### 2.2.1 January flight events with ANC airport <a class="anchor" id="2.2.1"></a>
[Back to top](#table)

1. Display all the flight events in January 2015 with five columns (Month, Origin
Airport, Destination Airport, Distance, and Arrival Delay), where the origin airport
'ANC' and name this dataframe as janFlightEventsAncDf.

In [19]:
# Filter the flight event that occurred in month = 1, year = 2015 and the origin airport ANC
janFlightEventsAncDf = flightsDf.filter((flightsDf.MONTH ==1) 
                                        & (flightsDf.YEAR ==2015) 
                                        & (flightsDf.ORIGIN_AIRPORT == 'ANC'))

# Select only five columns (Month, Origin Airport, Destination Airport, Distance, and Arrival Delay)
janFlightEventsAncDf = janFlightEventsAncDf.select('MONTH','ORIGIN_AIRPORT','DESTINATION_AIRPORT','DISTANCE','ARRIVAL_DELAY')

# Show the result
janFlightEventsAncDf.show()

+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261|           -6|
|    1|   

### 2.2.2 Average Arrival Delay From Origin to Destination <a class="anchor" id="2.2.2"></a>
[Back to top](#table)

2. From the query results on query no.1, please display a new query. Then please group
by ‘ORIGIN_AIRPORT’ AND ‘DESTINATION_AIRPORT’. Add a new column and
name it as ‘AVERAGE_DELAY’. This column value is the average from all
‘ARRIVAL_DELAY’ values. Then sort it based on ‘AVERAGE_DELAY’. Please
name this dataframe as janFlightEventsAncAvgDf.

In [20]:
# Import pyspark.sql.functions 
import pyspark.sql.functions as F

# Group by the result by ORIGIN_AIRPORT and DESTINATION_AIRPORT and aggregate average arrival delay
janFlightEventsAncAvgDf = janFlightEventsAncDf.groupBy(
    F.col('ORIGIN_AIRPORT'),F.col('DESTINATION_AIRPORT')).agg(
    F.avg('ARRIVAL_DELAY').alias('AVERAGE_DELAY'))

# Sort the result based on average delay
janFlightEventsAncAvgDf = janFlightEventsAncAvgDf.sort(janFlightEventsAncAvgDf.AVERAGE_DELAY)

# Show the result
janFlightEventsAncAvgDf.show()

+--------------+-------------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|
+--------------+-------------------+-------------------+
|           ANC|                ADK|              -27.0|
|           ANC|                HNL|              -20.0|
|           ANC|                MSP|             -19.25|
|           ANC|                BET| -9.090909090909092|
|           ANC|                SEA| -6.490196078431373|
|           ANC|                BRW| -4.333333333333333|
|           ANC|                OME|               -3.0|
|           ANC|                ADQ|-2.6666666666666665|
|           ANC|                CDV|                1.0|
|           ANC|                OTZ|               1.25|
|           ANC|                PHX|                2.0|
|           ANC|                DEN| 3.3333333333333335|
|           ANC|                PDX|                3.5|
|           ANC|                JNU|                5.0|
|           ANC|               

### 2.2.3 Join Query with Airports DataFrame <a class="anchor" id="2.2.3"></a>
[Back to top](#table)

3. Join the results on query no. 2 janFlightEventsAncAvgDf and airportsDf using inner
join operation. You may name this dataset as joinedSqlDf

In [21]:
# Inner Join janFlightEventsAncAvgDf and airportsD
joinedSqlDf = janFlightEventsAncAvgDf.join(airportsDf,
                                           janFlightEventsAncAvgDf.ORIGIN_AIRPORT == airportsDf.IATA_CODE,
                                           how ='inner')

joinedSqlDf.show()

+--------------+-------------------+-------------------+---------+--------------------+---------+-----+-------+--------+----------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|IATA_CODE|             AIRPORT|     CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+--------------+-------------------+-------------------+---------+--------------------+---------+-----+-------+--------+----------+
|           ANC|                BRW| -4.333333333333333|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                ADK|              -27.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                OME|               -3.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                JNU|                5.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                LAS|                9.0|      ANC|Ted Steven

## 2.3. Analysis <a class="anchor" id="2.3"></a>
[Back to top](#table)

In this section, we want to analyse when the delays are most likely to happen using Spark
SQL. By obtaining the day of week and month in all history of flight, implement the
following queries:



In [22]:
# Create Views from Dataframes
flightsDf.createOrReplaceTempView("sql_flights")
airportsDf.createOrReplaceTempView("sql_airports")

### 2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights <a class="anchor" id="2.3.1"></a>
[Back to top](#table)

1. Find the total number of flights events, total time delay and average of arrival delay
for each day of week (‘DAY_OF_WEEK’) sorted by the value of NumOfFlights in
descending order. This query represents the summary of all 2015 flights. What can
you analyse from this query results?


In [23]:
sql_task2_3_1 = spark.sql('''
  SELECT DAY_OF_WEEK,avg(ARRIVAL_DELAY) as MeanArrivalDelay, sum(ARRIVAL_DELAY) as TotalTimeDelay,
  count(FLIGHT_NUMBER) as NumOfFlights
  FROM sql_flights
  WHERE year = 2015
  GROUP BY DAY_OF_WEEK
  ORDER BY count(FLIGHT_NUMBER) desc

''')

sql_task2_3_1.show()

+-----------+------------------+--------------+------------+
|DAY_OF_WEEK|  MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----------+------------------+--------------+------------+
|          4| 5.684831897201573|        490186|       87683|
|          1| 5.883000999381335|        494478|       86317|
|          5| 4.715112525093624|        401638|       86253|
|          3|3.9745505431431147|        335150|       85607|
|          2| 4.391518272706391|        363262|       84449|
|          7| 4.299206488272548|        343498|       81422|
|          6| 1.813841449342257|        125750|       70453|
+-----------+------------------+--------------+------------+



#### Answer

Day of week 1 (Monday) and 4(Thursday) have the highest number of flights as well as highest number of total time delay and mean arrival delay, while day of week 6 (Saturday) has the least in every shown measurement.


### 2.3.2 Display mean arrival delay each month <a class="anchor" id="2.3.2"></a>
[Back to top](#table)

2. Find the average of arrival delay, total time delay, and total number of flight events for
each month (‘MONTH’) sorted by MeanArrivalDelay in ascending order (default).
What can you analyse from this query results?


In [24]:
sql_task2_3_2 = spark.sql('''
  SELECT MONTH,avg(ARRIVAL_DELAY) as MeanArrivalDelay,
  count(FLIGHT_NUMBER) as NumOfFlights, sum(ARRIVAL_DELAY) as TotalTimeDelay
  FROM sql_flights
  GROUP BY MONTH
  ORDER BY avg(ARRIVAL_DELAY) 

''')

sql_task2_3_2.show()

+-----+-------------------+------------+--------------+
|MONTH|   MeanArrivalDelay|NumOfFlights|TotalTimeDelay|
+-----+-------------------+------------+--------------+
|    9|-0.8498676252179341|       46733|        -39484|
|   10| -0.541989784312509|       48680|        -26209|
|   11| 0.8313745860658399|       46809|         38412|
|    4|  3.173803944339603|       48810|        153044|
|    5| 4.7121097658084405|       49691|        230785|
|    8|  4.713893233866763|       50524|        235063|
|    3|  5.011173860427592|       50816|        248454|
|    1|  5.804357298474946|       47136|        266420|
|   12|   6.15837046195826|       47866|        288883|
|    7|  6.786093552465234|       52065|        348907|
|    2|  8.123906203913085|       42798|        330513|
|    6|  9.747630090727856|       50256|        479174|
+-----+-------------------+------------+--------------+



#### Answer

Flights on Month 9(September) and month 10(October) seemed to have earlier schdeduled time than determined arrival time due to the negative value of total time delay and mean arrival delay, while on month 2 (Feburary) and month 6 (June) were the month that had the highest mean arrival delay


### 2.3.3 Relationship between mean departure delay and mean arrival delay <a class="anchor" id="2.3.3"></a>
[Back to top](#table)

3. Display the mean departure delay (MeanDeptDelay) and mean arrival delay
(MeanArrivalDelay) for each month (‘MONTH’) sorted by MeanDeptDelay in
descending order. What you can analyse from the relationship between two columns:
Mean Departure Delay and Mean Arrival Delay?

In [25]:
sql_task2_3_3 = spark.sql('''
  SELECT MONTH,avg(DEPARTURE_DELAY) as MeanDeptDelay,avg(ARRIVAL_DELAY) as MeanArrivalDelay
  FROM sql_flights
  GROUP BY MONTH
  ORDER BY avg(DEPARTURE_DELAY) desc

''')

sql_task2_3_3.show(20)

+-----+------------------+-------------------+
|MONTH|     MeanDeptDelay|   MeanArrivalDelay|
+-----+------------------+-------------------+
|    6|  13.9730063585922|  9.747630090727856|
|   12|11.821651454043728|   6.15837046195826|
|    7|11.708608758020432|  6.786093552465234|
|    2|11.620796080832823|  8.123906203913085|
|    8|10.086906141367324|  4.713893233866763|
|    1|  9.75401499511029|  5.804357298474946|
|    3| 9.718308159530178|  5.011173860427592|
|    5| 9.550310180006102| 4.7121097658084405|
|    4| 7.737554783759199|  3.173803944339603|
|   11| 6.630585898709037| 0.8313745860658399|
|   10| 5.243436261558784| -0.541989784312509|
|    9| 4.728506981740065|-0.8498676252179341|
+-----+------------------+-------------------+



#### Answer

The month that had high mean departure delay,especially in month 6(June), seemed to have high mean arrival delay as well. On the contrary, month 9(September) and 10 (October), which had low  mean departure delay seemed to have low arrival delay


# 3 RDDs vs DataFrame vs Spark SQL <a class="anchor" id="3"></a>


Implement the following queries using RDDs, DataFrames and SparkSQL separately. Log the time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the performance difference of these 3 approaches.

<strong>Find the MONTH and DAY_OF_WEEK, number of flights, and average delay where TAIL_NUMBER = ‘N407AS’. Note number of flights and average delay should be aggregated separately. The average delay should be grouped by both MONTH and DAYS_OF_WEEK.</strong>

## 3.1 RDD Operation<a class="anchor" id="3.1"></a>
[Back to top](#table)

In [26]:
%%time

# Filter the flight event that TAIL_NUMBER = N407AS
rdd_task3_1 = flight_rdd.filter(lambda x: x[6] == 'N407AS')

# Create tuple that tuple of 'TAIL_NUMBER', 'MONTH', 'DAY_OF_WEEK' acts as a key and 1 (Count the number of flights),
# DEPARTURE_DELAY and ARRIVAL_DELAY acts as a value
rdd_task3_1 = rdd_task3_1.map(lambda x:((x[6],x[1],x[3]),(1,x[11],x[22])) )

# Group by 'TAIL_NUMBER', 'MONTH', 'DAY_OF_WEEK' and sum all values
rdd_task3_1 = rdd_task3_1.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1],x[2]+y[2]))

# Transform the columns to 'NumOfFlights','MeanDeptDelay','MeanArrivalDelay'
rdd_task3_1 = rdd_task3_1.mapValues(lambda x : (x[0],x[1]/x[0],x[2]/x[0]))

# Unpack the tuple into list
rdd_task3_1 = rdd_task3_1.map(lambda x: list(x[0]) + list(x[1]) )

# Print the column headers
print('The column headers are ')
print([flight_header[6],flight_header[1],flight_header[3],'NumOfFlights','MeanDeptDelay','MeanArrivalDelay'])

# Show the result
rdd_task3_1.take(50)

The column headers are 
['TAIL_NUMBER', 'MONTH', 'DAY_OF_WEEK', 'NumOfFlights', 'MeanDeptDelay', 'MeanArrivalDelay']
CPU times: user 41.2 ms, sys: 4.59 ms, total: 45.8 ms
Wall time: 3.41 s


[['N407AS', 7, 3, 3, -0.3333333333333333, -5.333333333333333],
 ['N407AS', 12, 2, 2, 1.0, -11.5],
 ['N407AS', 10, 4, 1, -11.0, -6.0],
 ['N407AS', 6, 4, 1, -6.0, -15.0],
 ['N407AS', 8, 5, 3, 1.6666666666666667, -10.0],
 ['N407AS', 5, 6, 2, 0.5, -3.0],
 ['N407AS', 2, 3, 2, -12.5, -11.5],
 ['N407AS', 6, 2, 1, 33.0, 35.0],
 ['N407AS', 3, 5, 3, 5.666666666666667, 6.666666666666667],
 ['N407AS', 7, 5, 1, 9.0, -4.0],
 ['N407AS', 1, 3, 1, -7.0, -27.0],
 ['N407AS', 10, 5, 3, -6.333333333333333, -3.6666666666666665],
 ['N407AS', 12, 3, 2, 1.5, 18.5],
 ['N407AS', 8, 2, 2, -4.0, -11.0],
 ['N407AS', 11, 7, 3, -5.0, -4.0],
 ['N407AS', 9, 1, 2, -5.5, -15.5],
 ['N407AS', 3, 3, 1, 28.0, 3.0],
 ['N407AS', 1, 5, 2, -6.0, -21.0],
 ['N407AS', 12, 1, 1, -5.0, -1.0],
 ['N407AS', 4, 4, 3, 1.6666666666666667, -0.6666666666666666],
 ['N407AS', 8, 4, 1, -7.0, -5.0],
 ['N407AS', 5, 7, 3, 4.666666666666667, -7.666666666666667],
 ['N407AS', 2, 2, 2, -3.5, -9.5],
 ['N407AS', 11, 1, 1, 57.0, 35.0],
 ['N407AS', 8, 3, 

## 3.2 DataFrame Operation<a class="anchor" id="3.2"></a>
[Back to top](#table)

In [27]:
%%time

# Import pyspark.sql.functions 
# import pyspark.sql.functions as F

# Filter the flight event that TAIL_NUMBER = N407AS
df_task3_2  = flightsDf.filter(flightsDf.TAIL_NUMBER  == 'N407AS')

# Group By 'TAIL_NUMBER', 'MONTH', 'DAY_OF_WEEK' and aggregate all measurement
df_task3_2  = df_task3_2.groupBy(F.col('TAIL_NUMBER'),
                                 F.col('MONTH'),
                                 F.col('DAY_OF_WEEK')).agg(F.count('FLIGHT_NUMBER').alias('NumOfFlights'),
                                                           F.avg('DEPARTURE_DELAY').alias('MeanDeptDelay'),
                                                          F.avg('ARRIVAL_DELAY').alias('MeanArrivalDelay'))

# Sort by MeanArrivalDelay
df_task3_2 = df_task3_2.sort(df_task3_2.MONTH,df_task3_2.DAY_OF_WEEK)

# Show the result
df_task3_2.show(50)




+-----------+-----+-----------+------------+-------------------+-------------------+
|TAIL_NUMBER|MONTH|DAY_OF_WEEK|NumOfFlights|      MeanDeptDelay|   MeanArrivalDelay|
+-----------+-----+-----------+------------+-------------------+-------------------+
|     N407AS|    1|          1|           1|                4.0|               -6.0|
|     N407AS|    1|          2|           2|               12.5|               17.5|
|     N407AS|    1|          3|           1|               -7.0|              -27.0|
|     N407AS|    1|          5|           2|               -6.0|              -21.0|
|     N407AS|    1|          6|           3|  8.666666666666666|  4.333333333333333|
|     N407AS|    2|          1|           2|               -4.0|               -2.5|
|     N407AS|    2|          2|           2|               -3.5|               -9.5|
|     N407AS|    2|          3|           2|              -12.5|              -11.5|
|     N407AS|    2|          4|           2|               -8.5| 

## 3.3 Spark SQL OPERATION<a class="anchor" id="3.3"></a>
[Back to top](#table)

In [28]:
%%time

# Write the SQL command
sql_task3_3 = spark.sql('''
  SELECT TAIL_NUMBER,MONTH,DAY_OF_WEEK,
  count(FLIGHT_NUMBER) as NumOfFlights,
  avg(DEPARTURE_DELAY) as MeanDeptDelay, avg(ARRIVAL_DELAY) as MeanArrivalDelay
  FROM sql_flights
  WHERE TAIL_NUMBER = 'N407AS'
  GROUP BY TAIL_NUMBER,MONTH,DAY_OF_WEEK
  ORDER BY MONTH,DAY_OF_WEEK

''')

# Show the result
sql_task3_3.show(50)

+-----------+-----+-----------+------------+-------------------+-------------------+
|TAIL_NUMBER|MONTH|DAY_OF_WEEK|NumOfFlights|      MeanDeptDelay|   MeanArrivalDelay|
+-----------+-----+-----------+------------+-------------------+-------------------+
|     N407AS|    1|          1|           1|                4.0|               -6.0|
|     N407AS|    1|          2|           2|               12.5|               17.5|
|     N407AS|    1|          3|           1|               -7.0|              -27.0|
|     N407AS|    1|          5|           2|               -6.0|              -21.0|
|     N407AS|    1|          6|           3|  8.666666666666666|  4.333333333333333|
|     N407AS|    2|          1|           2|               -4.0|               -2.5|
|     N407AS|    2|          2|           2|               -3.5|               -9.5|
|     N407AS|    2|          3|           2|              -12.5|              -11.5|
|     N407AS|    2|          4|           2|               -8.5| 

## 3.4 Discussion<a class="anchor" id="3.4"></a>
[Back to top](#table)

From the %%time used for experiment above, it can be found that Spark SQL used the least total CPU times followed by dataframe operation and RDD consumed the longest CPU times. The link below gives us comparison between RDD, dataframe and SQL saying DF performs the best following by SQL and RDD performs worst

Reference
https://www.analyticsvidhya.com/blog/2020/11/what-is-the-difference-between-rdds-dataframes-and-datasets/